In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_text

from sklearn.model_selection import train_test_split
#from sklearn.inspection.permutation_importance import permutation_importance

import xgboost as xgb

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

pandas_version = pd.__version__

#skipTimEconsumming = True
skipTimEconsumming = False

print( 'Pandas version = ' + pandas_version + '\n' )

df = pd.read_csv('data-ch06-housing.csv')

df.head

Pandas version = 2.1.0



<bound method NDFrame.head of        longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -122.23     37.88                41.0        880.0           129.0   
1        -122.22     37.86                21.0       7099.0          1106.0   
2        -122.24     37.85                52.0       1467.0           190.0   
3        -122.25     37.85                52.0       1274.0           235.0   
4        -122.25     37.85                52.0       1627.0           280.0   
...          ...       ...                 ...          ...             ...   
20635    -121.09     39.48                25.0       1665.0           374.0   
20636    -121.21     39.49                18.0        697.0           150.0   
20637    -121.22     39.43                17.0       2254.0           485.0   
20638    -121.32     39.43                18.0       1860.0           409.0   
20639    -121.24     39.37                16.0       2785.0           616.0   

       population  ho

First, keep only the records where ocean_proximity is either '<1H OCEAN' or 'INLAND'

In [2]:
filtered_df = df[ ( df['ocean_proximity'] == '<1H OCEAN' ) | ( df['ocean_proximity'] == 'INLAND' ) ]

filtered_df['ocean_proximity'] = filtered_df['ocean_proximity'].replace('<1H OCEAN','less 1H OCEAN')

/tmp/ipykernel_19346/1448140692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['ocean_proximity'] = filtered_df['ocean_proximity'].replace('<1H OCEAN','less 1H OCEAN')


Preparation:

    Fill missing values with zeros.
    Apply the log transform to median_house_value.
    Do train/validation/test split with 60%/20%/20% distribution.
    Use the train_test_split function and set the random_state parameter to 1.
    Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [3]:
filtered_df = filtered_df.fillna(0)

seed = 1
df_full_train, df_test = train_test_split(filtered_df, test_size=0.2, random_state=seed)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=seed)

# Split done 

y_train_orig = df_train.median_house_value.values
y_val_orig = df_val.median_house_value.values
y_test_orig = df_test.median_house_value.values

y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [4]:
train_dicts = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [5]:
dt = DecisionTreeRegressor(max_depth=1)
#dt = RandomForestRegressor(max_depth=1)
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [6]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [7]:
y_train_pred = dt.predict(X_train)
print("RMSE on TRAIN data %s => ", rmse(y_train, y_train_pred) )

RMSE on TRAIN data %s =>  0.4522449592423713


In [8]:
y_val_pred = dt.predict(X_val)
print("RMSE on VALIDATION data %s => ", rmse(y_val, y_val_pred) )

RMSE on VALIDATION data %s =>  0.45168599736547244


Question 1

Let's train a decision tree regressor to predict the median_house_value variable.

    Train a model with max_depth=1.

Which feature is used for splitting the data?

    ocean_proximity
    total_rooms
    latitude
    population

AMSWER: ocean_proximity

In [9]:
featureNames=list(dv.get_feature_names_out())
print(featureNames)
print(export_text(dt, feature_names=featureNames))

['households', 'housing_median_age', 'latitude', 'longitude', 'median_income', 'ocean_proximity=INLAND', 'ocean_proximity=less 1H OCEAN', 'population', 'total_bedrooms', 'total_rooms']
|--- ocean_proximity=less 1H OCEAN <= 0.50
|   |--- value: [11.61]
|--- ocean_proximity=less 1H OCEAN >  0.50
|   |--- value: [12.30]



Train a random forest model with these parameters:

    n_estimators=10
    random_state=1
    n_jobs=-1 (optional - to make training faster)

What's the RMSE of this model on validation?

    0.045
    0.245
    0.545
    0.845

Answer: 0.245

In [10]:
dt = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
dt.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [11]:
y_train_pred = dt.predict(X_train)
print("RMSE on TRAIN data %s => ", round( rmse(y_train, y_train_pred), 3) )

y_val_pred = dt.predict(X_val)
print("RMSE on VALIDATION data %s => ", round( rmse(y_val, y_val_pred), 3) )

RMSE on TRAIN data %s =>  0.099
RMSE on VALIDATION data %s =>  0.245


Question 3

Now let's experiment with the n_estimators parameter

    Try different values of this parameter from 10 to 200 with step 10.
    Set random_state to 1.
    Evaluate the model on the validation dataset.

After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for retrieving the answer.

    10
    25
    50
    160

ANSWER: 160

In [12]:
if not skipTimEconsumming:
    for n_estimators in range(10,210,10):
        dt = RandomForestRegressor(n_estimators=n_estimators, random_state=1, n_jobs=-1)
        dt.fit(X_train, y_train)
    
        y_val_pred = dt.predict(X_val)
        print("n_estimators = ", n_estimators, "RMSE on VALIDATION data %s => ", round( rmse(y_val, y_val_pred), 3) )

n_estimators =  10 RMSE on VALIDATION data %s =>  0.245
n_estimators =  20 RMSE on VALIDATION data %s =>  0.239
n_estimators =  30 RMSE on VALIDATION data %s =>  0.237
n_estimators =  40 RMSE on VALIDATION data %s =>  0.235
n_estimators =  50 RMSE on VALIDATION data %s =>  0.235
n_estimators =  60 RMSE on VALIDATION data %s =>  0.234
n_estimators =  70 RMSE on VALIDATION data %s =>  0.234
n_estimators =  80 RMSE on VALIDATION data %s =>  0.234
n_estimators =  90 RMSE on VALIDATION data %s =>  0.234
n_estimators =  100 RMSE on VALIDATION data %s =>  0.234
n_estimators =  110 RMSE on VALIDATION data %s =>  0.234
n_estimators =  120 RMSE on VALIDATION data %s =>  0.234
n_estimators =  130 RMSE on VALIDATION data %s =>  0.234
n_estimators =  140 RMSE on VALIDATION data %s =>  0.234
n_estimators =  150 RMSE on VALIDATION data %s =>  0.233
n_estimators =  160 RMSE on VALIDATION data %s =>  0.233
n_estimators =  170 RMSE on VALIDATION data %s =>  0.233
n_estimators =  180 RMSE on VALIDATION d

Question 4

Let's select the best max_depth:

    Try different values of max_depth: [10, 15, 20, 25]
    For each of these values,
        try different values of n_estimators from 10 till 200 (with step 10)
        calculate the mean RMSE
    Fix the random seed: random_state=1

What's the best max_depth, using the mean RMSE?

    10
    15
    20
    25

ANSWER: 25

In [13]:
maxDepthList = [10, 15, 20, 25]

if not skipTimEconsumming:
    for maxDepth in maxDepthList:
        for n_estimators in range(10,210,10):
            dt = RandomForestRegressor(max_depth=maxDepth, n_estimators=n_estimators, random_state=1, n_jobs=-1)
            dt.fit(X_train, y_train)
    
            y_val_pred = dt.predict(X_val)
            print("maxDepth = ", maxDepth, "n_estimators = ", n_estimators, "RMSE on VALIDATION data %s => ", round( rmse(y_val, y_val_pred), 3) )

maxDepth =  10 n_estimators =  10 RMSE on VALIDATION data %s =>  0.251
maxDepth =  10 n_estimators =  20 RMSE on VALIDATION data %s =>  0.248
maxDepth =  10 n_estimators =  30 RMSE on VALIDATION data %s =>  0.246
maxDepth =  10 n_estimators =  40 RMSE on VALIDATION data %s =>  0.245
maxDepth =  10 n_estimators =  50 RMSE on VALIDATION data %s =>  0.245
maxDepth =  10 n_estimators =  60 RMSE on VALIDATION data %s =>  0.245
maxDepth =  10 n_estimators =  70 RMSE on VALIDATION data %s =>  0.245
maxDepth =  10 n_estimators =  80 RMSE on VALIDATION data %s =>  0.246
maxDepth =  10 n_estimators =  90 RMSE on VALIDATION data %s =>  0.246
maxDepth =  10 n_estimators =  100 RMSE on VALIDATION data %s =>  0.245
maxDepth =  10 n_estimators =  110 RMSE on VALIDATION data %s =>  0.245
maxDepth =  10 n_estimators =  120 RMSE on VALIDATION data %s =>  0.245
maxDepth =  10 n_estimators =  130 RMSE on VALIDATION data %s =>  0.245
maxDepth =  10 n_estimators =  140 RMSE on VALIDATION data %s =>  0.245
m

Question 5

We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorithm, it finds the best split. When doing it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

    Train the model with these parameters:
        n_estimators=10,
        max_depth=20,
        random_state=1,
        n_jobs=-1 (optional)
    Get the feature importance information from this model

What's the most important feature (among these 4)?

    total_rooms
    median_income
    total_bedrooms
    longitude

ANSWER: median_income

In [14]:
#from sklearn.feature_selection import SelectFromModel

rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)

rf.fit(X_train, y_train)
y_val_pred = rf.predict(X_val)

importances = rf.feature_importances_
featureNames = list(dv.get_feature_names_out())

forestImportances = pd.Series(importances, index=featureNames)

#print( forest_importances )
#print ("\n")
print( forestImportances.sort_values(ascending=False) )

median_income                    0.335724
ocean_proximity=INLAND           0.184371
ocean_proximity=less 1H OCEAN    0.181909
latitude                         0.102333
longitude                        0.085935
housing_median_age               0.029955
population                       0.027790
total_rooms                      0.020913
total_bedrooms                   0.016146
households                       0.014925
dtype: float64


Question 6

Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

    Install XGBoost
    Create DMatrix for train and validation
    Create a watchlist
    Train a model with these parameters for 100 rounds:

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

Now change eta from 0.3 to 0.1.

Which eta leads to the best RMSE score on the validation dataset?

    0.3
    0.1
    Both give equal value

ANSWER: 0.3

In [15]:

features = dv.get_feature_names_out().tolist()
print( features )
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)



['households', 'housing_median_age', 'latitude', 'longitude', 'median_income', 'ocean_proximity=INLAND', 'ocean_proximity=less 1H OCEAN', 'population', 'total_bedrooms', 'total_rooms']


In [16]:
eta = 0.3

xgb_params = {
    'eta': eta, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dtrain, num_boost_round=10)

y_val_pred = model.predict(dval)

print( "rmse for eta =", eta, " ", round( rmse(y_val, y_val_pred), 3) )


rmse for eta = 0.3   0.254


In [17]:
eta = 0.1

xgb_params = {
    'eta': eta, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dtrain, num_boost_round=10)

y_val_pred = model.predict(dval)

print( "rmse for eta =", eta, " ", round( rmse(y_val, y_val_pred), 3) )


rmse for eta = 0.1   0.323
